In [ ]:
using Plots
using LinearAlgebra
using Printf
using LaTeXStrings
using Optim
using BenchmarkTools
using Random
using ForwardDiff
using Dierckx

In [ ]:
default(lw=2,markersize = 6,
    xtickfont=font(12), ytickfont=font(12), 
    guidefont=font(14), legendfont=font(12),titlefont=font(12))

In [ ]:
include("bvps1d.jl"); # include source file with matrix discretizations

In [ ]:
n = 50; # set
x = LinRange(0,1,n+1)[2:end-1];
Δx = x[2] - x[1];
x_mid = LinRange(0,1,n+1)[2:end] .- Δx;
A = assemble_spmatrix(n, Δx); # matrix discretization of d^2/dx^2


In [ ]:
function solve_ode(f)
    return A\f;
end

function evaluate_u(x, u, x_new)
    Spline1D([0;x;1], [0; u; 0],k=3)(x_new);
end

In [ ]:
# a_true = 0.5;
# z_true = @. exp(-a_true * x);

p_true = 2;

f(p) = cos.(π * p * x);

f_true = f(p_true);


u_true = solve_ode(f_true);
plot(x, u_true)

Random.seed!(10);
# x_data = sort(rand(25));
x_data = [0.3, 0.45, 0.8];
u_data = Spline1D([0; x; 1], [0; u_true; 0],k=3)(x_data)

Random.seed!(100)
γ = 1e-2;
y_data = @. u_data + γ * randn();
plot(x, u_true)
scatter!(x_data, y_data)

In [ ]:
function F(p; λ = 1e-4)
    u = solve_ode(f(p[1]));
    u_predict = evaluate_u(x, u, x_data);
    res = .5 * mean(abs2, u_predict-y_data) + .5 * λ * p[1]^2;
    return res
end

In [ ]:
p0= [0.];
opt = optimize(F, p0)


In [ ]:
@show p_true;
@show opt.minimizer[1];